In [9]:
%pip install pandas
%pip install matplotlib
%pip install neuralforecast



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import logging
import matplotlib.pyplot as plt
import pandas as pd
from utilsforecast.plotting import plot_series
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS
from neuralforecast.utils import AirPassengersDF
df_transaction = pd.read_csv("/Users/hetapatel/Datathon 3-28-25/consumer_spending_ml/forecast_data/transaction_fact_20250325.csv")
df_transaction['transaction_date'] = pd.to_datetime(df_transaction['transaction_date'])
df_transaction['transaction_date'] = df_transaction['transaction_date'].dt.date
df_accountdim = pd.read_csv("/Users/hetapatel/Datathon 3-28-25/consumer_spending_ml/forecast_data/account_dim_20250325.csv")

In [11]:
feature_columns_transaction = ['transaction_date', 'current_account_nbr', 'transaction_amt', 'transaction_type']
df_transaction_filtered = df_transaction[feature_columns_transaction]
df_transaction_filtered = df_transaction_filtered[df_transaction_filtered['transaction_type'] == 'SALE']
df_accountdim.head()
df_transaction_filtered.head()

,transaction_date,current_account_nbr,transaction_amt,transaction_type
0,2024-06-05,X7jfKh6xrPAB8Tx6,15.78,SALE
1,2024-06-19,yntD77AZDylS48Q4,14.85,SALE
2,2024-06-26,LIJPI0sK28Pa7fX2,136.16,SALE
3,2024-08-15,CMAr5Apxwdzpvoze,8.74,SALE
4,2024-08-17,eJSfTCGPvJulGzd3,26.65,SALE


In [12]:
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx, NHITS

# Prepare data in the required format
df_forecast = df_transaction_filtered.groupby(['current_account_nbr', 'transaction_date'])['transaction_amt'].sum().reset_index()

# Convert date column to datetime
df_forecast['transaction_date'] = pd.to_datetime(df_forecast['transaction_date'])

# Rename columns to match NeuralForecast requirements
df_forecast = df_forecast.rename(columns={
    'transaction_date': 'ds',
    'transaction_amt': 'y',
    'current_account_nbr': 'unique_id'
})

# Initialize model - predict 92 days (Q4)
model = NeuralForecast(
    models=[NBEATSx(input_size=365, h=92, start_padding_enabled=True)],  # Look back 15 months, predict ~3 months (Q4)
    freq='D'
)

model.fit(df_forecast)
forecast = model.predict()

print("Forecast shape:", forecast.shape)
print("\nFirst few predictions:")
print(forecast.head())

# To get total Q4 spending per account
q4_spending = forecast.groupby('unique_id')['NBEATSx'].sum()
print("\nPredicted Q4 spending by account:")
print(q4_spending)

Seed set to 1
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 3.4 M  | train
-------------------------------------------------------
3.4 M     Trainable params
84.5 K    Non-trainable params
3.4 M     Total params
13.740    Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Epoch 2:  64%|██████▍   | 242/379 [00:06<00:03, 39.17it/s, v_num=32, train_loss_step=58.70, train_loss_epoch=65.50]

`Trainer.fit` stopped: `max_steps=1000` reached.


Epoch 2:  64%|██████▍   | 242/379 [00:06<00:03, 39.16it/s, v_num=32, train_loss_step=58.70, train_loss_epoch=65.50]


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 379/379 [00:01<00:00, 281.03it/s]
Forecast shape: (1112924, 3)

First few predictions:
          unique_id         ds    NBEATSx
0  00gwujJkd597VxLd 2025-03-21  27.825865
1  00gwujJkd597VxLd 2025-03-22  27.807743
2  00gwujJkd597VxLd 2025-03-23  27.149782
3  00gwujJkd597VxLd 2025-03-24  27.537836
4  00gwujJkd597VxLd 2025-03-25  26.481491

Predicted Q4 spending by account:
unique_id
00gwujJkd597VxLd     2694.118408
00iP5U82D8XwVQ9G      465.240479
00oyr3QppAzjLws4     6151.034180
02WRVV8yusgcZWLW    12478.448242
033o9yHYen3xoz6k     3979.056641
                        ...     
zyZhjzJwhpOgSvmc     2541.898682
zz3nbtZXS41NZk0h     3776.042236
zzBy2qNM78aRV580     3160.462891
zzEuUBBmvGiVnabb     9529.258789
zzR9PvG7dY9u5iHU    13486.830078
Name: NBEATSx, Length: 12097, dtype: float32


In [13]:
# Define columns we want to keep
feature_columns_account = ['current_account_nbr']
df_accountdim_filtered = df_accountdim[feature_columns_account]

# Add year and quarter columns
df_transaction_filtered['transaction_date'] = pd.to_datetime(df_transaction_filtered['transaction_date'])
df_transaction_filtered['year'] = df_transaction_filtered['transaction_date'].dt.year
df_transaction_filtered['quarter'] = df_transaction_filtered['transaction_date'].dt.quarter

# Calculate spending by year and quarter
quarterly_spending = df_transaction_filtered.groupby(
    ['current_account_nbr', 'year', 'quarter']
)['transaction_amt'].sum().reset_index()

# Create separate dataframes for 2024 and 2025
spending_2024 = quarterly_spending[quarterly_spending['year'] == 2024].copy()
spending_2025 = quarterly_spending[quarterly_spending['year'] == 2025].copy()

# Pivot 2024 data
spending_2024_wide = spending_2024.pivot(
    index='current_account_nbr',
    columns='quarter',
    values='transaction_amt'
).reset_index()
spending_2024_wide = spending_2024_wide.rename(columns={
    1: 'q1_2024_spending',
    2: 'q2_2024_spending',
    3: 'q3_2024_spending',
    4: 'q4_2024_spending'
})

# Get Q1 2025 data
spending_2025_q1 = spending_2025[spending_2025['quarter'] == 1].copy()
spending_2025_q1 = spending_2025_q1[['current_account_nbr', 'transaction_amt']]
spending_2025_q1 = spending_2025_q1.rename(columns={'transaction_amt': 'q1_2025_spending'})

# Get Q4 2025 predictions
q4_predictions = forecast.groupby('unique_id')['NBEATSx'].sum().reset_index()
q4_predictions = q4_predictions.rename(columns={'NBEATSx': 'predicted_q4_2025_spending'})

# Merge everything together
final_df = df_accountdim_filtered.merge(
    spending_2024_wide,
    on='current_account_nbr',
    how='left'
).merge(
    spending_2025_q1,
    on='current_account_nbr',
    how='left'
).merge(
    q4_predictions,
    left_on='current_account_nbr',
    right_on='unique_id',
    how='left'
)

# Clean up
final_df = final_df.drop('unique_id', axis=1, errors='ignore')
final_df = final_df.fillna(0)

# Save to CSV
final_df.to_csv('quarterly_spending_predictions.csv', index=False)

print("First few rows with quarterly spending and Q4 2025 predictions:")
print(final_df.head())

First few rows with quarterly spending and Q4 2025 predictions:
  current_account_nbr  q1_2024_spending  q2_2024_spending  q3_2024_spending  \
0    nTgalbpd5Rsol7CA               0.0              0.00             95.00   
1    xYaLrD9IvmbVl1PB               0.0              0.00           1318.90   
2    JM78WNziiRWgmL8e               0.0            617.15            311.20   
3    bv4NLVhWTGCf8KgC               0.0              0.00            182.65   
4    nrXceQabYIHo31lp               0.0              0.00              0.00   

   q4_2024_spending  q1_2025_spending  predicted_q4_2025_spending  
0              0.00            557.29                12461.628906  
1            788.88             94.16                25715.671875  
2            784.30              0.00                 4120.576660  
3              0.00              0.00                  576.967896  
4              0.00              0.00                    0.000000  


In [15]:
account_id = 'nTgalbpd5Rsol7CA'
if account_id in q4_spending:
    print(f"Predicted Q4 spending for account {account_id}: ${q4_spending[account_id]:.2f}")
else:
    print(f"Account {account_id} not found in predictions")

print("Available account IDs:")
print(q4_spending.head())

Predicted Q4 spending for account nTgalbpd5Rsol7CA: $12461.63
Available account IDs:
unique_id
00gwujJkd597VxLd     2694.118408
00iP5U82D8XwVQ9G      465.240479
00oyr3QppAzjLws4     6151.034180
02WRVV8yusgcZWLW    12478.448242
033o9yHYen3xoz6k     3979.056641
Name: NBEATSx, dtype: float32
